In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

# Load datasets
calendar = pd.read_csv('/kaggle/input/calender/calendar.csv')
sell_prices = pd.read_csv('/kaggle/input/sell-prices/sell_prices.csv')
sales_train_eval = pd.read_csv('/kaggle/input/sales-train-eval/sales_train_evaluation.csv')
sales_train_valid = pd.read_csv('/kaggle/input/sales-train-valid/sales_train_validation.csv')

# Store-by-Store Merge

In [ ]:
import gc
import pandas as pd
import numpy as np

def reduce_memory_usage(df):
    """ Reduce memory usage of a dataframe by downcasting numerical types where possible. """
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type not in [object, 'category']:
            c_min, c_max = df[col].min(), df[col].max()
            if str(col_type).startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                df[col] = df[col].astype(np.float32)
        else:
            if col_type == object and df[col].nunique() / len(df[col]) < 0.5:
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"  Memory reduced: {start_mem:.2f} MB -> {end_mem:.2f} MB "
          f"(Reduced by {(start_mem - end_mem)/start_mem*100:.1f}%)")
    return df

def process_store(store, sales_eval, calendar, sell_prices, output_dir):
    """ Process data for a single store: filter, melt, merge, reduce memory, and save. """
    print(f"\nProcessing store: {store}")

    # Step 1: Filter sales data for the store
    df_store = sales_eval[sales_eval["store_id"] == store].copy()
    df_store = reduce_memory_usage(df_store)

    # Step 2: Melt the subset
    fixed_cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]
    date_cols = [c for c in df_store.columns if c.startswith("d_")]
    df_melted_sub = pd.melt(df_store, id_vars=fixed_cols, value_vars=date_cols, var_name="d", value_name="sales")
    del df_store
    gc.collect()

    # Step 3: Merge with calendar
    df_cal_sub = pd.merge(df_melted_sub, calendar, how="left", on="d")
    del df_melted_sub
    gc.collect()

    # Step 4: Filter and merge sell prices
    sp_sub = sell_prices[sell_prices["store_id"] == store].copy()
    sp_sub = reduce_memory_usage(sp_sub)
    df_merged_sub = pd.merge(df_cal_sub, sp_sub, how="left", on=["store_id", "item_id", "wm_yr_wk"])
    del df_cal_sub, sp_sub
    gc.collect()

    # Step 5: Reduce memory usage again
    df_merged_sub = reduce_memory_usage(df_merged_sub)

    # Step 6: Save to disk
    out_path = f"{output_dir}/merged_{store}.pkl"
    df_merged_sub.to_pickle(out_path)
    print(f"  Saved merged data for store={store}, shape={df_merged_sub.shape} -> {out_path}")

    # Step 7: Clear memory
    del df_merged_sub
    gc.collect()

# Make sure dataframes are loaded before calling this
output_directory = "/kaggle/working"

all_stores = sales_train_eval["store_id"].unique()
print("Found store_ids:", all_stores)

for store in all_stores:
    process_store(store, sales_train_eval, calendar, sell_prices, output_directory)

# EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
# Loop over each store dataset
for store in all_stores:
    pickle_path = f"/kaggle/working/merged_{store}.pkl"
    
    if not os.path.exists(pickle_path):
        print(f"No file for {store}, skipping.")
        continue

    print(f"\nProcessing {store}...")
    df_merged = pd.read_pickle(pickle_path)

    # Sample the data for all visualizations (10% of the data)
    df_sampled = df_merged.sample(frac=0.1, random_state=42)
    
    # Basic info about the dataframe
    print(df_sampled.info())
    
    
    # Check the first few rows to understand the structure
    print(df_sampled.head())
    # Check missing values
    missing_values = df_sampled.isnull().sum()
    print("Missing values per column:\n", missing_values)

    # visualize missing values

    plt.figure(figsize=(10, 6))
    sns.heatmap(df_sampled.isnull(), cbar=False, cmap='viridis')
    plt.title(f"Missing Values Heatmap for {store}")
    plt.savefig(f'missing_vals_heatmap_{store}.png', dpi=300)
    plt.show()

    # Summary statistics
    print(df_sampled.describe())
    # Visualize the distribution of numerical columns
    # Numerical columns for distribution
    numerical_cols = ['sales', 'sell_price', 'wm_yr_wk', 'month', 'wday']

    for col in numerical_cols:
        plt.figure(figsize=(10, 6))
        sns.histplot(df_sampled[col], kde=True, bins=20)
        plt.title(f"Distribution of {col} for {store}")
        plt.savefig(f'distribution_of_{col}_for_{store}.png', dpi=300)
        plt.show()

    # Categorical columns for distribution
    categorical_cols = ['weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']

    for col in categorical_cols:
        plt.figure(figsize=(10, 6))
        sns.countplot(data=df_sampled, x=col)
        plt.title(f"Distribution of {col} for {store}")
        plt.xticks(rotation=45)
        plt.savefig(f'distribution_of_{col}_for_{store}.png', dpi=300)
        plt.show()

    

    # Encode categorical columns for correlation heatmap
    df_encoded = df_sampled.copy()
    le = LabelEncoder()
    for col in categorical_cols:
        df_encoded[col] = le.fit_transform(df_encoded[col])

    # Compute correlation matrix for numerical columns
    correlation_matrix = df_encoded[numerical_cols + categorical_cols].corr()

    # Plot heatmap of the correlation matrix
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title(f"Correlation Matrix for {store}")
    plt.savefig(f'corr_matrix_for_{store}.png', dpi=300)
    plt.show()

    # Time series plot for 'sales'
    plt.figure(figsize=(14, 7))
    df_sampled['date'] = pd.to_datetime(df_sampled['date'])
    df_sampled.groupby('date')['sales'].sum().plot()
    plt.title(f"Sales Time Series for {store}")
    plt.xlabel("Date")
    plt.ylabel("Total Sales")
    plt.savefig(f"time_series_for_{store}.png", dpi=300)
    plt.show()

    # Boxplot to detect outliers in sales data
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=df_sampled['sales'])
    plt.title(f"Outliers in Sales for {store}")
    plt.savefig(f'outliers_in_sales_for_{store}.png', dpi=300)
    plt.show()


# Compare sales trends across stores using sampled data
plt.figure(figsize=(14, 7))
for store in all_stores:
    store_data = pd.read_pickle(f"/kaggle/working/merged_{store}.pkl")
    
    # Sample the data for faster processing
    store_data_sampled = store_data.sample(frac=0.1, random_state=42)
    
    store_data_sampled['date'] = pd.to_datetime(store_data_sampled['date'])
    store_data_sampled.groupby('date')['sales'].sum().plot(label=store)

plt.legend()
plt.title("Sales Trends Across Stores (Sampled Data)")
plt.xlabel("Date")
plt.ylabel("Total Sales")
plt.savefig(f'sales_trends_across_stores_sampled.png', dpi=300)
plt.show()



